In [20]:
from imutils import contours
import cv2
import numpy as np
import keras
import os, shutil
import imutils
import sys
import argparse

In [36]:
cropped_dataset_dir="./cropped_image/"
original_dataset_dir="./img/"
ocr_image="./ocr_a_reference.png"

In [22]:
image_name_list=os.listdir(original_dataset_dir)

In [23]:
def find_rotated_rect(colorimg):
    img=cv2.cvtColor(colorimg,cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(img, 45, 133)
    kernel = np.ones((5,5), np.uint8)
    closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
    contours,hierarchy = cv2.findContours(closing, 1, 2)
    cnt=find_largest_erea(contours)
    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    return box,rect

In [24]:
def find_largest_erea(contours):
    largest_eara=0
    largest_eara_index=0
    for cnt in contours:
        if cv2.contourArea(cnt)>largest_eara:
            largest_eara=cv2.contourArea(cnt)
            largest_eara_contours=cnt
            
    return largest_eara_contours

In [25]:
def draw_straight_regtangle(original_image):
    gray_img=cv2.cvtColor(colorimg,cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray_img, 45, 133)
    kernel = np.ones((5,5), np.uint8)
    closing = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
    (contours,_) = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt=find_largest_erea(contours)
    (x,y,w,h) = cv2.boundingRect(cnt)
    cv2.rectangle(original_image,(x,y),(x+w,y+h)   ,(255,0,0), 2)
    return (x,y,w,h)

In [26]:
def find_red_regtangle(image):
    B, G, R = cv2.split(image)
    ret,thresh = cv2.threshold(R, 245, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    (contours,_) = cv2.findContours(closing, 1, 2)
    cnt=find_largest_erea(contours)
    (x,y,w,h) = cv2.boundingRect(cnt)
    cv2.rectangle(image,(x,y),(x+w,y+h)   ,(255,0,0), 2)
    return (x,y,w,h)

In [39]:
def crop_rotate_image(colorimg):
   
    
    colorimg= cv2.resize(colorimg, None, fx=0.25, fy=0.25)
    box,rect=find_rotated_rect(colorimg)
    colorimg = cv2.drawContours(colorimg,[box],0,(0,0,255),3)
    height, width = colorimg.shape[:2]
    if(rect[1][0]>rect[1][1]):
#         print(count_angle(box[0],box[1]))
        rotated = imutils.rotate_bound(colorimg,  np.int_(count_angle(box[0],box[1])))
    else:
        rotated = imutils.rotate_bound(colorimg, np.int_(count_angle(box[0],box[1])-90))
    (x,y,w,h)=find_red_regtangle(rotated)
    cropped=rotated[y:y+h,x:x+w]
#     cv2.imwrite(cropped_image+image_name,cropped)
    return cropped

In [40]:
def count_angle(a,b):
    a=np.array(a)
    b=np.array(b)
    c=np.array([a[0],b[1]])
    ab=b-a
    ac=c-a
    cosine_angle=np.dot(ab,ac)/(np.linalg.norm(ab)*np.linalg.norm(ac))
    angle=np.arccos(cosine_angle)
    return np.degrees(angle)

In [41]:
ref = cv2.imread(ocr_image)
ref = cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(ref, 70, 255, cv2.THRESH_BINARY_INV)[1]

In [42]:
refCnts = cv2.findContours(ref.copy(), cv2.RETR_EXTERNAL,
cv2.CHAIN_APPROX_SIMPLE)
refCnts = imutils.grab_contours(refCnts)
refCnts = contours.sort_contours(refCnts, method="left-to-right")[0]
digits = {}


In [43]:
for (i, c) in enumerate(refCnts):
    (x, y, w, h) = cv2.boundingRect(c)
    roi = ref[y:y + h, x:x + w]
    roi = cv2.resize(roi, (57, 88))
    digits[i] = roi

In [44]:
sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4 , 3))



In [45]:
def recogenize_card_number(image):
    image = imutils.resize(image, width=300)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray=(255-gray)
    tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, 
                              rectKernel)
    gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,ksize=-1)
    gradX = np.absolute(gradX)
    (minVal, maxVal) = (np.min(gradX), np.max(gradX))
    gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
    gradX = gradX.astype("uint8")
    gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
    thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                      
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    locs = []
    for (i, c) in enumerate(cnts):
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)
        if ar > 2.5 and ar < 4.0:
            if (w > 40 and w < 55) and (h > 10 and h < 20):
                locs.append((x, y, w, h))
    locs = sorted(locs, key=lambda x:x[0])
    output = []
    for (i, (gX, gY, gW, gH)) in enumerate(locs):
        # initialize the list of group digits
        groupOutput = []
        group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
        group = cv2.threshold(group, 0, 255,
            cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]  
#         cv2.imshow("group",group)
#         cv2.waitKey(0)
        digitCnts = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        digitCnts = imutils.grab_contours(digitCnts)
        digitCnts = contours.sort_contours(digitCnts,method="left-to-right")[0]
        for c in digitCnts:
            (x, y, w, h) = cv2.boundingRect(c)
            roi = group[y:y + h, x:x + w]
            roi = cv2.resize(roi, (57, 88))
            scores = []
            for (digit, digitROI) in digits.items():
                result = cv2.matchTemplate(roi, digitROI,
                    cv2.TM_CCOEFF)
                (_, score, _, _) = cv2.minMaxLoc(result)
                scores.append(score)
            groupOutput.append(str(np.argmax(scores)))
        cv2.rectangle(image, (gX - 5, gY - 5),
            (gX + gW + 5, gY + gH + 5), (0, 0, 255), 2)
        cv2.putText(image, "".join(groupOutput), (gX, gY - 15),
            cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
        output.extend(groupOutput)
        print("Credit Card #: {}".format("".join(output)))
    return image

In [47]:
image_name_list=os.listdir(original_dataset_dir)
for i in range(0,len(image_name_list),20):
    image = cv2.imread(original_dataset_dir+image_name_list[i])
    cropped=crop_rotate_image(image)
    cv2.imwrite(cropped_dataset_dir+image_name_list[i],cropped)
image_name_list=os.listdir(cropped_dataset_dir)
for i in range(len(image_name_list),20):
    cropped=cv2.imread(cropped_dataset_dir+image_name_list[i])
    image=recogenize_card_number(cropped)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

/home/yumi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [35]:
cv2.destroyAllWindows()



















In [ ]:
# for i in range(len(image_name_list)):
#     image = cv2.imread(original_dataset_dir+image_name_list[i])
#     image = imutils.resize(image, width=300)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     gray=(255-gray)
#     tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, 
#                               rectKernel)
#     gradX = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0,ksize=-1)
#     gradX = np.absolute(gradX)
#     (minVal, maxVal) = (np.min(gradX), np.max(gradX))
#     gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
#     gradX = gradX.astype("uint8")
#     gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel)
#     thresh = cv2.threshold(gradX, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
#     thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)
#     cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
#                         cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     locs = []
#     for (i, c) in enumerate(cnts):
#         (x, y, w, h) = cv2.boundingRect(c)
#         ar = w / float(h)
#         if ar > 2.5 and ar < 4.0:
#             if (w > 40 and w < 55) and (h > 10 and h < 20):
#                 locs.append((x, y, w, h))
#     locs = sorted(locs, key=lambda x:x[0])
#     output = []
#     for (i, (gX, gY, gW, gH)) in enumerate(locs):
#         # initialize the list of group digits
#         groupOutput = []
#         group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
#         group = cv2.threshold(group, 0, 255,
#             cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]  
#         digitCnts = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#         digitCnts = imutils.grab_contours(digitCnts)
#         digitCnts = contours.sort_contours(digitCnts,method="left-to-right")[0]
#         for c in digitCnts:
#             (x, y, w, h) = cv2.boundingRect(c)
#             roi = group[y:y + h, x:x + w]
#             roi = cv2.resize(roi, (57, 88))
#             scores = []
#             for (digit, digitROI) in digits.items():
#                 result = cv2.matchTemplate(roi, digitROI,
#                     cv2.TM_CCOEFF)
#                 (_, score, _, _) = cv2.minMaxLoc(result)
#                 scores.append(score)
#             groupOutput.append(str(np.argmax(scores)))
#         cv2.rectangle(image, (gX - 5, gY - 5),
#             (gX + gW + 5, gY + gH + 5), (0, 0, 255), 2)
#         cv2.putText(image, "".join(groupOutput), (gX, gY - 15),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
#         output.extend(groupOutput)
#         print("Credit Card #: {}".format("".join(output)))
#     cv2.imshow("Image", image)
#     cv2.waitKey(0)
